<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100" align="right" /> 

# Cours TAL - Laboratoire 3<br/>*Depedency parser* pour le français dans spaCy

**Objectif**

Évaluer l'analyseur syntaxique en dépendances fourni par spaCy dans le modèle `fr_core_news_sm`, puis le comparer avec un analyseur entraîné par vous-mêmes.  Les données sont les mêmes qu'au Labo 2 et la démarche du labo est similaire aussi.

## 1. Prise en main de l'analyseur de spaCy

In [1]:
import pandas as pd

In [2]:
import spacy
nlp = spacy.load("fr_core_news_sm") # charge la pipeline

**1a.** Pour la pipeline `fr_core_news_sm`, veuillez afficher les traitements disponibles, puis désactiver tous les traitements sauf `tok2vec`, `morphologizer` et `parser`, puis vérifiez que la désactivation a bien fonctionné.

In [3]:
print(f"Liste des traitements disponibles : {nlp.pipe_names}")

for pipe in nlp.pipe_names:
    if pipe not in ["tok2vec", "morphologizer", "parser"]:
        nlp.disable_pipe(pipe)

print(f"Liste des traitements actifs : {nlp.pipe_names}")

Liste des traitements disponibles : ['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
Liste des traitements actifs : ['tok2vec', 'morphologizer', 'parser']


In [4]:
from spacy.lang.fr.examples import sentences
s1 = sentences[2] # prenons la 3e phrase comme exemple

**1b.** Veuillez analyser `s1` avec la pipeline `nlp` puis afficher chaque token, son POS tag, et son étiquette indiquant la relation de dépendance (entre crochets, après le token).  Quelle information essentielle manque dans cette représentation ?

Note : le *morphologizer* fournit aussi les POS tags.  La liste des tags possibles est [fournie par spaCy](https://spacy.io/models/fr#fr_core_news_md-labels).  

In [5]:
s1_doc = nlp(s1)

print("Tokens:")
for token in s1_doc:
    print(f"{token.text} [{token.pos_} | {token.dep_}]")

Tokens:
San [PRON | nsubj]
Francisco [PROPN | flat:name]
envisage [VERB | ROOT]
d' [ADP | case]
interdire [NOUN | obl:arg]
les [DET | det]
robots [NOUN | obj]
coursiers [ADJ | amod]
sur [ADP | case]
les [DET | det]
trottoirs [NOUN | nmod]


> L'étiquette indiquant la relation de dépendance est affichée, mais rien n'indique de quels mots ils sont dépendants.

**1c.** Veuillez afficher tous les groupes de mots qui sont soit des `nsubj` soit des `obj` dans la phrase `s1` (c'est à dire les sujets et les objets du verbe).   Indication : le sous-arbre d'un token *t* est accessible comme `t.subtree`. 

In [6]:
print("Groupes de mots étant soit 'nsubj' soit 'obj':")

for token in s1_doc:
    if token.dep_ in ["nsubj", "obj"]:
        print(f"- {' '.join([t.text for t in token.subtree])} [{token.dep_}: {token.text}]")

Groupes de mots étant soit 'nsubj' soit 'obj':
- San Francisco [nsubj: San]
- les robots coursiers sur les trottoirs [obj: robots]


## 2. Évaluation quantitative de l'analyseur sur une phrase 

Les données sont les mêmes que celles du Labo 2.  Vous les avez déjà transformées au Labo 2 dans un format utilisable par spaCy, dans un dossier nommé `Labo2/spacy_data` que vous allez réutiliser.  Les trois fichiers contiennent des phrases en français annotées aussi avec les arbres de dépendance.  Le fichier `fr-ud-train.conllu` est destiné à l'entraînement, `fr-ud-dev.conllu` au réglage des paramètres, et `fr-ud-test.conllu` à l'évaluation finale.

**2a.** En inspectant un des fichiers d'origine avec un éditeur texte, veuillez indiquer dans quelles colonnes se trouvent les informations sur les relations de dépendance, et comment elles sont représentées.

> En inspectant simplement le fichier `fr-ud-dev.conllu`, il peut être difficile de déterminer quelles colonnes concernent les relations de dépendances.
>
> Selon la [documentation officielle du format](https://universaldependencies.org/format.html), les relations de dépendances sont situées dans la 8e colonne (DEPREL) et sont représentées avec le [format des étiquettes de dépendances universelles](https://universaldependencies.org/u/dep/index.html), c'est à dire qu'elles sont sélectionnées parmis les 37 relations syntaxiques universelles.
>
> La 7e colonne (HEAD) contient l'identifiant du token gouverneur (ou 0 si c'est la racine) et la 9e colonne (DEPS) contient un graphe de dépendances amélioré sous forme de paires (HEAD, DEPREL) ou un `_` si cela n'est pas renseigné.

In [7]:
from spacy.tokens import DocBin, Doc
test_data = DocBin().from_disk("../Labo2/spacy_data/fr-ud-test.spacy")
# for doc in test_data.get_docs(nlp.vocab):  # exemple
#     for sent in doc.sents:
#         print(sent)

**2b**. On rapplle que les données des fichiers convertis peuvent être chargées dans un objet de type `DocBin`.  Ici, un tel objet contient un ensemble de documents, chacun contenant 10 phrases.  Chaque document est un objet de type `Doc`.  Le code donné ci-dessous vous permet de charger les données de test et vous montre comment les afficher.

* Veuillez stocker la *7e phrase du 2e document des données de test* dans une variable nommée `s2`.
* Veuillez afficher cette phrase (elle commence par "Trois ans").

In [8]:
docs = list(test_data.get_docs(nlp.vocab))
s2 = list(docs[1].sents)[6]

print(f"7e phrase du 2e document :\t{s2}")

7e phrase du 2e document :	Trois ans plus tard, il tient un discours sur la crise.


**2c.** En utilisant `displaCy` comme expliqué [ici](https://spacy.io/usage/visualizers) veuillez afficher graphiquement l'arbre de dépendances de la phrase `s2` tel qu'il est fourni dans les données.  Pour être affichée, la phrase doit être transformée en objet `Doc`.

In [9]:
from spacy import displacy

In [10]:
# Il n'est pas nécessaire de transformer s2 en doc, car dans notre cas la phrase peut être affichée sans.
# Nous avons tout de même effectué la transformation, afin de suivre la consigne de la question.
displacy.render(s2.as_doc(), style="dep", options = {"compact": True, "distance": 90})

**2d.** En utilisant `displaCy`, veuillez également afficher l'arbre de dépendances calculé par la pipeline `nlp` pour cette même phrase `s2`.  Pour être analysée et affichée, la phrase doit être transformée en objet `Doc`.

In [11]:
displacy.render(nlp(s2.as_doc()), style="dep", options = {"compact": True, "distance": 90})

**2e.** Veuillez comparer les deux arbres de dépendances et indiquer ici les différences.  Quel est le taux de correction de la pipeline `nlp` sur cette phrase ?

Suggestion : il peut être utile de sauvegarder les deux arbres dans des images SVG, en écrivant dans un fichier le résultat retourné par `displacy.render` avec l'option `jupyter = False`.

> La phrase utilisée contient 13 tokens, dont 2 de ponctuations, pour un total de 10 relations entre les tokens. À noter que les deux tokens de ponctuations sont liés au token précédent, sans générer de relation.
> 
> La pipeline nlp génère également 10 relations, dont 2 sont différentes des relations correctes :
> - Une relation existe de `plus` vers `ans` (obl:mod), alors que `ans` devrait dépendre du verbe `tient` (obl).
> - Une relation existe de `tient` vers `tard` (advmod), alors que `tard` devrait dépendre de `ans` (advmod).
>
> Sur les 13 tokens, 2 sont donc mal classifiés par la pipeline nlp, ce qui donne donc un taux de correction de 2/13 ≈ 0.154. Si l'on cherche le score obtenu pour les éléments bien classifiés, le score est alors de 11/13 ≈ 0.846.

**2f.**  Veuillez appliquer le `Scorer` de spaCy (voir Labo 2) et afficher les deux scores qu'il produit pour l'analyse en dépendances (avec trois décimales après la virgule).  Retrouvez-vous les scores de la question précédente ? Pourquoi ?

In [12]:
from spacy.scorer import Scorer
from spacy.training import Example

In [13]:
score = Scorer().score_deps([Example(nlp(s2.as_doc()), s2.as_doc())], "dep")

print(f"UAS: {score['dep_uas']:.3f}")
print(f"LAS: {score['dep_las']:.3f}")

UAS: 0.846
LAS: 0.846


> Le taux de correction de la question précédente ne peut pas être retrouvé tel quel, puisque les scores UAS et LAS calcule la qualité d'alignement entre les deux éléments, en comparant les prédictions avec les annotations correctes. Cependant, ces scores correspondent bien à notre score, puisqu'il s'agit de la valeur obtenue lorsque l'on prends en considération que 11 annotations sur les 13 sont correctes, offrant donc un score de ~ 0.846.

## 3. Évaluation du *dependency parser* de `fr_core_news_sm` sur l'ensemble des phrases test

**3a.** Veuillez calculer les deux scores qui caractérisent l'analyseur en dépendances de la pipeline `nlp` sur toutes les données de test présentes dans `test_data`.  Comment se comparent ces scores avec ceux mentionnés [dans la documentation de fr_core_news_sm](https://spacy.io/models/fr#fr_core_news_sm) ?

In [14]:
score = Scorer().score(
    [Example(nlp(doc.copy()), doc) for doc in test_data.get_docs(nlp.vocab)]
)

print(f"UAS: {score['dep_uas']:.3f}")
print(f"LAS: {score['dep_las']:.3f}")

UAS: 0.816
LAS: 0.689


> Nous avons décidé de travailler en effectuant la copie des documents et non pas en les recréant à partir du vocabulaire. De cette manière, les annotations ont pu être conservées pour les documents de prédictions.
>
> La documentation indique un score UAS (DEP_UAS) de 0.88 et un score LAS (DEP_LAS) de 0.84 . Dans un cas comme dans l'autre, nos valeurs sont inférieures aux scores de référence, en particulier pour le score LAS. Le score UAS est relativement proche du score de référence, indiquant une annotation raisonnablement correcte, malgré un peu plus d'erreurs que dans le cas de référence. Ces différences peuvent être attribuées à l'entrainement du modèle sur un corpus différent de celui testé ici, les relations de dépendance pouvant alors ne pas être attribuées de la bonne manière. La séparation de notre texte en un ensemble de documents peut en être partiellement la cause, si cela fait perdre du contexte significatif aux différentes phrases. Le score LAS, quant à lui, montre une baisse significative par rapport au score de référence, indiquant des problèmes plus importants pour l'attribution des types de dépendances. Le labelling étant plus précis, il semble normal d'obtenir un score plus bas si le modèle n'a pas suffisamment d'informations contextuelles pour attribuer correctement les relations de dépendance ou si celles-ci sont ambiguës.

**3b.** Le *scorer* fournit également des scores détaillés pour chaque type de relation de dépendances.  Veuillez afficher ces valeurs dans un tableau proprement formaté, trié par score F1 décroissant, avec trois décimales.

In [15]:
df = pd.DataFrame.from_dict(score["dep_las_per_type"], orient='index')
df.columns = ['Precision', 'Recall', 'F1']
df.sort_values(by='F1', ascending=False, inplace=True)

pd.options.display.float_format = "{:.3f}".format
df

,Precision,Recall,F1
det,0.938,0.937,0.938
case,0.874,0.927,0.899
root,1.000,0.786,0.880
cc,0.802,0.802,0.802
mark,0.745,0.797,0.770
nsubj,0.807,0.719,0.760
aux:pass,0.636,0.925,0.754
nsubj:pass,0.662,0.860,0.748
cop,0.738,0.748,0.743
nummod,0.755,0.719,0.737


## 4. Entraîner puis évaluer un nouveau *parser* français dans spaCy

Le but de cette partie est d'entraîner une pipeline spaCy pour le français sur les données de `fr-ud-train.conllu`, puis de comparer le modèle obtenu avec le modèle prêt-à-l'emploi testé au point précédent (voir le Labo 2 et les [instructions de spaCy](https://spacy.io/usage/training#quickstart)).

**4a.** Paramétrage de l'entraînement :
* générez un fichier de départ grâce à [l'interface web](https://spacy.io/usage/training#quickstart), en indiquant que vous gardez seulement les composants `morphologizer` et `parser` dans la pipeline ;
* sauvegardez le code généré par spaCy dans un fichier local `base_config.cfg` ;
* générez un fichier `config.cfg` sur votre ordinateur en exécutant la ligne de commande suivante. 

In [16]:
!python -m spacy init fill-config base_config.cfg config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Veuillez effectuer l'entraînement avec la ligne de commande suivante.  Faites plusieurs essais, d'abord avec un petit nombre d'époques (*à indiquer dans config.cfg*), pour estimer le temps nécessaire et observer les messages affichés.  Augmentez progressivement le nombre d'époques, jusqu'à ce que les scores sur le jeu de validation n'augmentent plus (si vous avez le temps).  Pendant combien d'époques entraînez-vous au final ?

In [17]:
# Note : il vaut mieux exécuter cela directement dans une fenêtre de commande, pour voir les logs en temps réel.
# !python -m spacy train config.cfg \
#     --output ./myDEPparser1 \
#     --paths.train ../Labo2/spacy_data/fr-ud-train.spacy \
#     --paths.dev ../Labo2/spacy_data/fr-ud-dev.spacy \
#     --verbose

```
[2025-04-05 16:25:41,628] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
✔ Created output directory: myDEPparser1
ℹ Saving to output directory: myDEPparser1
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-04-05 16:25:45,046] [INFO] Set up nlp object from config
[2025-04-05 16:25:45,057] [DEBUG] Loading corpus from path: ..\Labo2\spacy_data\fr-ud-dev.spacy
[2025-04-05 16:25:45,059] [DEBUG] Loading corpus from path: ..\Labo2\spacy_data\fr-ud-train.spacy
[2025-04-05 16:25:45,059] [INFO] Pipeline: ['tok2vec', 'morphologizer', 'parser']
[2025-04-05 16:25:45,063] [INFO] Created vocabulary
[2025-04-05 16:25:45,063] [INFO] Finished initializing nlp object
[2025-04-05 16:26:10,819] [INFO] Initialized pipeline components: ['tok2vec', 'morphologizer', 'parser']
✔ Initialized pipeline

============================= Training pipeline =============================
[2025-04-05 16:26:10,831] [DEBUG] Loading corpus from path: ..\Labo2\spacy_data\fr-ud-dev.spacy
[2025-04-05 16:26:10,832] [DEBUG] Loading corpus from path: ..\Labo2\spacy_data\fr-ud-train.spacy
ℹ Pipeline: ['tok2vec', 'morphologizer', 'parser']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS MORPH...  LOSS PARSER  POS_ACC  MORPH_ACC  DEP_UAS  DEP_LAS  SENTS_F  SCORE
---  ------  ------------  -------------  -----------  -------  ---------  -------  -------  -------  ------
  0       0          0.00         224.53       504.38    34.04      29.37    24.62     9.44     0.15    0.24
  0     200       4076.83       19248.97     37619.19    87.07      82.56    71.50    63.42    81.56    0.76
  0     400       6106.26        9646.12     25713.28    90.24      87.90    77.30    70.88    92.09    0.82
  0     600       6043.18        7294.74     22102.46    91.29      89.39    78.45    72.95    94.25    0.83
  0     800       6622.88        6605.53     22011.44    91.85      90.37    80.19    74.98    92.14    0.84
  0    1000       6505.47        5656.93     19717.99    92.28      91.06    80.99    75.94    93.86    0.85
  0    1200       6554.39        5286.01     19127.81    92.39      91.41    81.90    76.99    95.13    0.86
  0    1400       6925.99        4947.77     19057.76    92.69      91.80    81.59    76.47    94.77    0.86
  1    1600       6795.02        4322.92     17605.95    92.81      92.00    82.00    77.39    94.99    0.86
  1    1800       7133.65        4014.09     16746.83    92.80      92.14    81.74    77.31    95.14    0.86
  1    2000       7413.99        4116.94     16786.62    92.82      92.07    82.60    78.11    95.66    0.86
  1    2200       7756.61        3850.70     16827.29    93.02      92.32    82.76    78.45    95.20    0.87
  1    2400       7941.20        4029.44     16722.92    93.06      92.48    83.43    79.40    94.86    0.87
  1    2600       8371.70        4018.36     17306.68    92.98      92.46    82.97    78.90    95.65    0.87
  1    2800       8407.93        4014.37     17000.82    93.19      92.73    83.83    79.61    96.02    0.87
  2    3000       8628.17        3813.38     16635.18    93.32      92.74    83.58    79.64    95.51    0.87
  2    3200       8374.46        3132.56     14904.19    93.24      92.73    83.69    79.38    95.34    0.87
  2    3400       8930.12        3361.20     15474.46    93.24      92.72    83.96    80.02    95.84    0.87
  2    3600       9263.96        3473.93     15434.55    93.36      92.90    83.85    80.05    96.16    0.88
  2    3800       9084.25        3290.00     14773.37    93.45      92.87    83.95    79.85    95.61    0.88
  2    4000       9602.68        3133.96     14988.53    93.40      92.98    84.16    80.39    96.35    0.88
  2    4200       9774.70        3302.76     15072.96    93.40      93.01    84.33    80.50    95.92    0.88
  3    4400       9770.63        3201.89     14441.56    93.45      93.08    84.29    80.54    96.05    0.88
  3    4600      10241.10        2941.22     14243.12    93.45      92.91    84.15    80.30    95.82    0.88
  3    4800      10506.24        2818.77     13832.77    93.40      93.01    84.15    80.13    95.83    0.88
  3    5000      11219.74        2863.98     14140.47    93.35      93.06    84.17    80.39    95.56    0.88
  3    5200      11126.38        2797.06     14193.04    93.43      93.02    84.50    80.66    96.24    0.88
  3    5400      11095.29        2849.36     13973.46    93.41      93.02    84.17    80.40    96.23    0.88
  3    5600      11454.07        2969.16     13729.25    93.44      93.07    84.43    80.68    95.37    0.88
  3    5800      12051.29        3018.24     14151.85    93.45      93.03    84.90    81.01    95.68    0.88
  4    6000      11405.89        2422.85     12918.35    93.48      92.99    85.13    81.42    95.92    0.88
  4    6200      12504.17        2626.70     13084.72    93.45      93.01    84.89    81.20    96.06    0.88
  4    6400      12737.20        2558.20     12908.75    93.45      93.04    84.81    80.96    95.31    0.88
  4    6600      13707.71        2654.45     13701.64    93.55      93.04    84.96    81.53    96.17    0.88
  4    6800      13596.57        2562.43     13251.81    93.47      93.13    84.91    81.18    96.29    0.88
  4    7000      13596.54        2689.95     13616.15    93.59      93.23    84.76    81.23    95.82    0.88
  4    7200      13955.75        2563.59     13263.38    93.55      93.14    84.72    81.15    95.96    0.88
  5    7400      13961.36        2520.17     12909.74    93.71      93.31    85.21    81.70    95.26    0.88
  5    7600      14314.13        2318.97     12145.28    93.59      93.16    85.04    81.59    96.36    0.88
  5    7800      15668.42        2380.80     12693.07    93.65      93.31    85.05    81.48    96.77    0.88
  5    8000      15879.24        2449.79     12810.25    93.55      93.14    84.71    81.35    96.37    0.88
  5    8200      16376.59        2475.15     13191.64    93.58      93.24    84.86    81.28    96.02    0.88
  5    8400      15816.04        2528.74     12304.19    93.63      93.28    85.20    81.78    95.97    0.88
  5    8600      15334.79        2391.78     11997.73    93.61      93.29    85.13    81.69    96.69    0.88
  6    8800      16372.70        2272.01     12571.86    93.54      93.28    85.01    81.44    96.16    0.88
  6    9000      16277.84        2131.20     11267.94    93.51      93.22    85.33    81.72    96.16    0.88
✔ Saved pipeline to output directory
myDEPparser1\model-last
```

> La configuration par défaut a été utilisée, avec un paramètre `patience` permettant un arrêt anticipé de l'entraînement et un nombre maximal d'étapes (`max_steps`) avant l'arret de l'entraînement. Un total de 6 epoques ont été réalisées avant que le modèle ne s'arrête, ayant atteint son paramètre de patience.

**4b.**  Veuillez charger le meilleur modèle (pipeline) dans la variable `nlp2` et afficher ses scores sur les données de test.  Comment se comparent les résultats avec les précédents ?

In [18]:
nlp2 = spacy.load("./myDEPparser2/model-best")
test_data = DocBin().from_disk("../Labo2/spacy_data/fr-ud-test.spacy")

score = Scorer().score(
    [Example(nlp2(doc.copy()), doc) for doc in test_data.get_docs(nlp2.vocab)]
)

print(f"UAS: {score['dep_uas']:.3f}")
print(f"LAS: {score['dep_las']:.3f}")

UAS: 0.882
LAS: 0.838


**4c.** Veuillez afficher les scores détaillés pour chaque type de relation de dépendances, dans un tableau formaté comme au 3b.

In [19]:
df = pd.DataFrame.from_dict(score["dep_las_per_type"], orient='index')
df.columns = ['Precision', 'Recall', 'F1']
df.sort_values(by='F1', ascending=False, inplace=True)

pd.options.display.float_format = "{:.3f}".format
df

,Precision,Recall,F1
det,0.972,0.984,0.978
aux:caus,1.000,0.923,0.960
case,0.926,0.962,0.943
root,1.000,0.877,0.935
aux,0.947,0.881,0.913
cop,0.885,0.885,0.885
cc,0.884,0.860,0.872
amod,0.857,0.875,0.866
mark,0.856,0.874,0.865
nummod,0.879,0.849,0.864


**4d.** Quels changements observez-vous en haut (3 premiers labels) et en bas du classement ?  Voyez-vous un label pour lequel les scores n'augmentent pas avec le parser entraîné ?

> Si l'on observe les classements obtenus, on constate que les trois premiers labels étaient `[det, case, root]` avec le modèle prêt-à-l'emploi et sont `[det, aux:caus, case]` pour le modèle entrainé, avec des f-scores plus élevés que ceux obtenus précédemment. En ce qui concerne les trois derniers labels, ceux-ci étaient initialement `[flat:foreign, discourse, csubj]` et sont maintenant devenu `[obl:agent, discourse, csubj]`, toujours avec des f-scores à 0.
>
> On notera toutefois qu'il y a moins de labels avec un f-score de 0 dans les résultats du modèle entrainé, indiquant une amélioration par rapport au modèle prêt-à-l'emploi et une potentielle précision plus élevée de notre modèle sur les relations de dépendances dans certains contextes spécifiques à notre corpus.
>
> csubj (clausal subject) est un label pour lequel les scores n'augmentent pas avec le parser entraîné, certainement parce que ce label n'est pas utilisé dans notre corpus (dans UD_english, ce label est utilisé pour 340 nodes, c'est 0%: https://universaldependencies.org/docs/en/dep/csubj.html)

**Fin du Labo.** Veuillez nettoyer ce notebook en gardant seulement les résultats désirés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.